# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [22]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
#print(sinasc.shape)
#sinasc.drop_duplicates().shape
# Não há duplicados

num_registros = sinasc.shape[0]

num_registros_sem_duplicados = sinasc.drop_duplicates().shape[0]

ha_duplicados = num_registros != num_registros_sem_duplicados

print("Número total de registros: {}".format(num_registros))

print("Número de registros não duplicados: {}".format(num_registros_sem_duplicados))

print("Há linhas duplicadas? {}".format('Sim' if ha_duplicados else 'Não'))

Número total de registros: 27028
Número de registros não duplicados: 27028
Há linhas duplicadas? Não


In [23]:
# 2) seu código aqui

missing_values = sinasc.isnull().sum()

print("Número de valores missing por variável:")
print(missing_values)


Número de valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [24]:
# 3) seu código aqui

colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_interesse = sinasc[colunas_interesse]

missing_values_interesse = sinasc_interesse.isnull().sum()

print("Número de valores missing por variável nas colunas selecionadas:")
print(missing_values_interesse)


Número de valores missing por variável nas colunas selecionadas:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [25]:
# 4) seu código aqui

print("Número total de registros antes da remoção:", sinasc.shape[0])

sinasc_apgar_preenchido = sinasc.dropna(subset=['Apgar5'])

print("Número de registros após remover Apgar5 não preenchido:", sinasc_apgar_preenchido.shape[0])

num_missing_apgar5 = sinasc_apgar_preenchido['Apgar5'].isnull().sum()
print(f"Número de valores ausentes em Apgar5 após remoção: {num_missing_apgar5}")

Número total de registros antes da remoção: 27028


KeyError: ['Apgar5']

In [26]:
# 5) seu código aqui

sinasc['ESTCIVMAE'].replace('9', pd.NA, inplace=True)
sinasc['CONSULTAS'].replace('9', pd.NA, inplace=True)

num_registros_sem_duplicados = sinasc.drop_duplicates().shape[0]

print(sinasc['ESTCIVMAE'])
print(sinasc['CONSULTAS'])

0        5.0
1        2.0
2        9.0
3        5.0
4        2.0
        ... 
27023    2.0
27024    1.0
27025    2.0
27026    2.0
27027    5.0
Name: ESTCIVMAE, Length: 27028, dtype: float64
0        4
1        4
2        4
3        3
4        4
        ..
27023    4
27024    4
27025    4
27026    4
27027    4
Name: CONSULTAS, Length: 27028, dtype: int64


In [27]:
# 6) Seu código aqui

sinasc['QTDFILVIVO'].fillna(0, inplace=True)

print(sinasc['QTDFILVIVO'])

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64


In [28]:
# 7) seu código aqui

# Substituir os valores faltantes pelas médias
sinasc['GESTACAO'].fillna(sinasc['GESTACAO'].mean(), inplace=True)
sinasc['GRAVIDEZ'].fillna(sinasc['GRAVIDEZ'].mean(), inplace=True)
sinasc['CONSULTAS'].fillna(sinasc['CONSULTAS'].mean(), inplace=True)
sinasc['APGAR5'].fillna(sinasc['APGAR5'].mean(), inplace=True)

print(sinasc['GESTACAO'])
print(sinasc['GRAVIDEZ'])
print(sinasc['CONSULTAS'])
print(sinasc['APGAR5'])
# Para cada variável restante ('GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'), a melhor opção para substituir os valores faltantes seria calcular e utilizar a média dos valores observados, dependendo da distribuição dos dados e do contexto específico do estudo realizado com os dados do SINASC. Isso ajudará a manter a integridade e a utilidade dos dados ao realizar análises subsequentes.


TypeError: can only concatenate str (not "int") to str

In [29]:
# 8) seu código aqui

intervalos = [-1, 3, 5, 7, 10]
categorias = ['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']

sinasc['Apgar_Categoria'] = pd.cut(sinasc['APGAR1'], bins=intervalos, labels=categorias, right=False)

frequencias = sinasc['Apgar_Categoria'].value_counts()

print("Frequências da categorização do Apgar:")
print(frequencias)


Frequências da categorização do Apgar:
normal              25543
asfixia leve          810
asfixia moderada      217
asfixia severa        130
Name: Apgar_Categoria, dtype: int64


In [30]:
# 9) seu código aqui

sinasc.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

print("Número total de registros:", sinasc.shape[0])
print("Número de registros não duplicados:", sinasc.drop_duplicates().shape[0])

print(sinasc.head())


Número total de registros: 27028
Número de registros não duplicados: 27028
   origem   codestab  codmunnasc  locnasc  idademae  estcivmae  \
0       1  2679477.0      110001        1        19        5.0   
1       1  2679477.0      110001        1        29        2.0   
2       1  2679477.0      110001        1        37        9.0   
3       1  2516500.0      110001        1        30        5.0   
4       1  2516500.0      110001        1        30        2.0   

            escmae  codocupmae  qtdfilvivo  qtdfilmort  ...  contador  \
0      8 a 11 anos         NaN         0.0         0.0  ...         1   
1      8 a 11 anos    999992.0         1.0         0.0  ...         2   
2      8 a 11 anos    513205.0         2.0         0.0  ...         3   
3  12 anos ou mais    231205.0         0.0         0.0  ...         4   
4      8 a 11 anos    999992.0         1.0         0.0  ...         5   

  munresstatus munrestipo               munresnome  munresuf munreslat  \
0        ATIVO 